Gravity Model Implementation 

In [2]:
import numpy as np
import pandas as pd

In [3]:
# Define the Cost Function
Cij = np.array([
    [0, 0, 50, 0, 0],
    [0, 0, 60, 0, 0],
    [0, 0, 0, 30, 0],
    [20, 0, 80, 0, 20],
    [0, 70, 90, 10, 0]
], dtype=np.float64)


In [4]:
# Replace zeros in Cij to avoid division/log errors
Cij[Cij == 0] = 1e-6

In [5]:
# Define Origin and Destination vectors
O = np.array([80, 150, 100, 160, 180])
D = np.array([100, 90, 320, 80, 300])


In [6]:
# Check and balance the vectors O and D
sum_O = np.sum(O)
sum_D = np.sum(D)


In [7]:
# Balancing the Origins and Destinations
if sum_O != sum_D:
    if sum_O < sum_D:
        correction_ratio = sum_D / sum_O  # Calculate correction ratio
        O = O * correction_ratio  # Adjust O by the correction ratio
    else:
        correction_ratio = sum_O / sum_D  # Calculate correction ratio
        D = D * correction_ratio  # Adjust D by the correction ratio

In [8]:
# Define the deterrence function
def fcij(Cij):
    alpha = 0.05
    beta = 0.2
    return alpha * np.exp(-beta * (np.log(Cij + 1))**2)

FCij = fcij(Cij)


In [9]:

# Gravity Model Implementation
def gravity_distribution(FCij, O, D, tol=1e-4, max_iter=100):
    n = len(O)
    A = np.ones(n)
    B = np.ones(n)
    T = np.zeros((n, n))
    T_total = np.sum(O)  # Total trips (same for O and D)

    
    for iteration in range(max_iter):
        # Update A_i
        for i in range(n):
            #A[i] = 1 / np.sum(B * D * FCij[i, :])
            A[i] = 1 / max(np.sum(B * D * FCij[i, :]), 1e-6)

        # Update B_j
        for j in range(n):
            #B[j] = 1 / np.sum(A * O * FCij[:, j])
            B[j] = 1 / max(np.sum(A * O * FCij[:, j]), 1e-6)

        # Calculate T_ij
        for i in range(n):
            for j in range(n):
                T[i, j] = A[i] * O[i] * B[j] * D[j] * FCij[i, j]
        
        # Error calculation based on the provided formula
        production_error = np.sum(np.abs(O - T.sum(axis=1)))
        attraction_error = np.sum(np.abs(D - T.sum(axis=0)))
        error_percentage = (production_error + attraction_error) / T_total * 100
        
        # Create DataFrame for displaying T
        T_df = pd.DataFrame(
            T,
            index=['A', 'B', 'C', 'D', 'E'],
            columns=['A', 'B', 'C', 'D', 'E']
        )
        
        # Add row and column summations
        T_df["Row Sum"] = T_df.sum(axis=1)  # Add row sums
        T_df.loc["Column Sum"] = T_df.sum(axis=0)  # Add column sums
        
        # Display the matrix
        print(f"Iteration {iteration + 1}:\n")
        print(T_df.round(2))
        print(f"\nError: {error_percentage:.6f}\n{'-'*50}")
        
        # Check for convergence
        if error_percentage < tol:
            break
    
    return T

In [10]:
# Run the gravity model
T = gravity_distribution(FCij, O, D)

# Display the final result
print("Final Trip Distribution Matrix with Row and Column Sums:\n")
final_output = pd.DataFrame(
    T, index=['A', 'B', 'C', 'D', 'E'], columns=['A', 'B', 'C', 'D', 'E']
)
final_output["Row Sum"] = final_output.sum(axis=1)  # Add row sums
final_output.loc["Column Sum"] = final_output.sum(axis=0)  # Add column sums
print(final_output.round(2))


Iteration 1:

                 A      B       C      D       E  Row Sum
A            13.19  10.29   12.55   9.07   39.57    84.68
B            24.89  19.42   17.76  17.12   74.66   153.84
C            11.79   9.20  246.99   0.77   35.36   304.10
D            10.09  50.26   28.37  44.31   30.27   163.31
E            40.05   0.83   14.34   8.72  120.14   184.07
Column Sum  100.00  90.00  320.00  80.00  300.00   890.00

Error: 38.487016
--------------------------------------------------
Iteration 2:

                 A      B       C      D       E  Row Sum
A            13.90  10.71   24.93   8.85   41.70   100.09
B            27.07  20.85   36.40  17.24   81.20   182.75
C             4.32   3.33  170.76   0.26   12.97   191.64
D            11.03  54.22   58.44  44.84   33.08   201.61
E            43.68   0.89   29.47   8.81  131.05   213.90
Column Sum  100.00  90.00  320.00  80.00  300.00   890.00

Error: 13.215132
--------------------------------------------------
Iteration 3:

        